In [2]:
import asyncio
import itertools
import datetime

import pandas

import pipdepgraph
import pipdepgraph.entrypoints
import pipdepgraph.entrypoints.common
from pipdepgraph.repositories import (
    direct_dependency_repository,
    known_package_name_repository,
)


In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
import collections

packages_in_violation = []
packages_processed = 0
async with pipdepgraph.entrypoints.common.initialize_async_connection_pool() as db_pool:
    dd_repo = direct_dependency_repository.DirectDependencyRepository(db_pool)
    kpn_repo = known_package_name_repository.KnownPackageNameRepository(db_pool)
    async for package_name in kpn_repo.iter_known_package_names(date_last_checked_before=datetime.datetime.now()):
        direct_dependencies: list[direct_dependency_repository.DirectDependencyResult] = []
        version_distribution_groups = collections.defaultdict(lambda: collections.defaultdict(set))
        async for dd_of_package in dd_repo.iter_direct_dependencies(kv_package_name=package_name.package_name, vd_package_type="bdist_wheel", vd_processed=True, output_as_dict=False):
            direct_dependencies.append(dd_of_package)
            version_distribution_groups[dd_of_package.known_version.package_version][dd_of_package.version_distribution.version_distribution_id].add((
                dd_of_package.direct_dependency.extras,
                dd_of_package.direct_dependency.dependency_name,
                dd_of_package.direct_dependency.dependency_extras,
                dd_of_package.direct_dependency.version_constraint,
            ))

        any_differences = False
        for version, distributions in version_distribution_groups.items():
            for d1, d2 in itertools.pairwise(distributions.values()):
                any_differences = True
                break
            if any_differences:
                break
        if any_differences:
            packages_in_violation.append(package_name)


In [7]:
len(packages_in_violation)

246

In [ ]:

async with pipdepgraph.entrypoints.common.initialize_async_connection_pool() as db_pool:
    dd_repo = direct_dependency_repository.DirectDependencyRepository(db_pool)
    kpn_repo = known_package_name_repository.KnownPackageNameRepository(db_pool)

    direct_dependencies: list[direct_dependency_repository.DirectDependencyResult] = []
    dds_by_vdid_lookup = collections.defaultdict(list)
    version_distribution_groups = collections.defaultdict(lambda: collections.defaultdict(set))
    async for dd_of_package in dd_repo.iter_direct_dependencies(kv_package_name='cryptography', vd_package_type="bdist_wheel", vd_processed=True, output_as_dict=False):
        direct_dependencies.append(dd_of_package)
        dds_by_vdid_lookup[dd_of_package.version_distribution.version_distribution_id].append(dd_of_package)

        version_distribution_groups[dd_of_package.known_version.package_version][dd_of_package.version_distribution.version_distribution_id].add((
            # dds_of_package.direct_dependency.extras,
            dd_of_package.direct_dependency.dependency_name,
            dd_of_package.direct_dependency.dependency_extras,
            dd_of_package.direct_dependency.version_constraint,
        ))

    for version, distributions in version_distribution_groups.items():
        for d1, d2 in itertools.pairwise(distributions.items()):
            if d1[1] != d2[1]:
                dds_1 = dds_by_vdid_lookup[d1[0]]
                dds_2 = dds_by_vdid_lookup[d2[0]]
